# RAGによるLLMへの知識付与

## ライブラリをインポート

In [2]:
import warnings

warnings.filterwarnings("ignore")

import re

import faiss
import numpy as np
import torch
from IPython.display import Markdown
from pypdf import PdfReader
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

## データを読み込み文章を一定のまとまり（チャンク）に分割し、前処理を実施

In [3]:
# PDFを読み込む関数
def load_pdf(file_path):
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text


# テキストをチャンクに分割する関数
def split_text(text, chunk_size=300):
    chunks = [text[i : i + chunk_size] for i in range(0, len(text), chunk_size)]
    return chunks


# PDFから抽出したテキストから改行コードを削除する関数
def clean_text(text):
    return text.replace("\n", "").strip()

In [4]:
# PDFファイルをロードしてチャンクに分割
file_path = "../input/MIハンドブック_はじめに.pdf"
text = load_pdf(file_path)
cleaned_text = clean_text(text)
documents = split_text(cleaned_text)

In [5]:
documents

['はじめに   良い材料を 早く創りたいので 、マテリアルズ・インフォマティクス（ MI）を活用して”いる”。このような声 はもう珍しくありません 。 MIブームに端を発して 、大学・企業 ともにMIの活用検討が 始められて 既にある程度の期間が 経過しました 。すでにMIに取り組んでいる 大学・企業 からはMI活用の成果 が多く報告されており 、MIは期待に応えられるということが共通理解になりつつあります。 MIとは？有効なものなのか？が大勢を占めていた数年前とは隔世の感があります。  本書を手に取っていただいた方は、 材料開発者として、 あるいは材料開発を支援する データ分析者として「MIを',
 '活用して 材料開発を加速したい。 」という気持ち だと考えています 。   著者の高原と福岡は 多くの民間企業などに対して MI活用の支援をさせて頂いています。 学生時代に は二人とも 材料工学を専攻 し、それぞれ別 メーカーでMIを活用した材料開発業務を経て、 現在はコンサルティング ・データ分析・講演・教育・データ活用システムの提供 といった 様々な立場 ・内容で活動を行って います。  MIの普及に伴い、様々な書籍やセミナーが 発刊・実施されています 。しかしそれは、MIとは？という概論か、 整形されたデータに対して分析アルゴリズムを適用する ものが主と感じています。  材料開発の実務の',
 '中で MIを活用するには、さらに踏み込んだ情報が必要です。 例えば、データの整形方法。 MIに限らず機械学習 ・データ分析の入門書では分析可能 な形に整形されたデータから分析がスタートします。一方で、 MIが普及していない 材料開発の現場では、 データ分析フレンドリーな 形でデータが蓄積されていません。そのため、そ のデータを分析可能にする手順が分からずMIの活用を断念してしまうことがあります 。 次にデータや分析結果の 見方という観点。 本書執筆の 時点では 、AIにデータを丸ごと投げ込み、出てきた結果をそのまま実験すればより良い材料が出来るということは 、ほとんどの場合 ありません 。手持',
 'ちのデータのどのような点に着目し、 データ分析を適用するか。出てきた分析結果を どのように 材料開発の加速に役立て ていくか。こういった ノウハウが必要に なります。

## Transformer系列のモデルを用いたベクトル化を行う

In [6]:
# SentenceTransformerを使用して埋め込みを生成
embedding_model = SentenceTransformer("tohoku-nlp/bert-base-japanese-char")
embeddings = embedding_model.encode(documents, convert_to_tensor=False)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
embeddings

array([[-0.00860138, -0.01544509,  0.05732762, ...,  0.10766538,
         0.02744278,  0.07329488],
       [-0.02874077, -0.03830459,  0.0481893 , ...,  0.10121736,
         0.02751632,  0.04565822],
       [-0.00059892, -0.09614192,  0.06609966, ...,  0.07873933,
        -0.14247668,  0.16064684],
       ...,
       [ 0.05802056, -0.14048971,  0.08772241, ...,  0.11404857,
        -0.02176722,  0.12419258],
       [ 0.050071  , -0.04091918, -0.02482079, ...,  0.0119293 ,
         0.0106582 ,  0.05920728],
       [ 0.02757211,  0.03278049,  0.03972042, ...,  0.04832372,
         0.0965587 , -0.30393988]], dtype=float32)

## 検索ができるようにベクトル化したテキストデータにインデックスを付与

In [8]:
# FAISSインデックスを作成
d = embeddings[0].shape[0]  # 埋め込みの次元
index = faiss.IndexFlatL2(d)
index.add(np.array(embeddings))

## LLMを読み込む

In [9]:
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-jpn-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-2b-jpn-it",
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## テキスト生成のための設定

In [10]:
# テキスト生成パイプラインを設定
text_gen_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [11]:
def ask_question(question):
    # 質問を埋め込みに変換
    question_embedding = embedding_model.encode([question], convert_to_tensor=False)

    # FAISSで最も近いチャンクを検索
    _, indices = index.search(np.array(question_embedding), k=1)
    retrieved_text = documents[indices[0][0]]

    # 質問と関連情報、回答の間に改行を追加
    input_text = f"質問: {question}\n\n関連情報: {retrieved_text}\n\n回答: "

    # パイプラインを使用して回答を生成
    outputs = text_gen_pipeline(input_text, max_new_tokens=256)

    # リストから文字列を取得
    assistant_response = outputs[0]["generated_text"].strip()

    return assistant_response

## RAGを使用せず知識付与を行わない状態でLLMに質問する

In [12]:
question = "MIとは何ですか？"

outputs = text_gen_pipeline(question, max_new_tokens=256)
response = outputs[0]["generated_text"].strip()
print(response)

MIとは何ですか？

「**AI**」の略称で、人工知能の技術です。

**AIとは？**

* **人間のような思考・判断能力を持つコンピュータ**
* **学習・分析・予測・行動**を自動化できる能力を持つ

**AIの活用例**

* **自動運転車**
* **医療診断**
* **顧客対応**
* **商品推薦**
* **翻訳**
* **画像認識**


In [13]:
display(Markdown(response))

MIとは何ですか？

「**AI**」の略称で、人工知能の技術です。

**AIとは？**

* **人間のような思考・判断能力を持つコンピュータ**
* **学習・分析・予測・行動**を自動化できる能力を持つ

**AIの活用例**

* **自動運転車**
* **医療診断**
* **顧客対応**
* **商品推薦**
* **翻訳**
* **画像認識**

## RAGを使用して知識付与を行った状態でLLMに質問する

In [14]:
print(ask_question(question))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

質問: MIとは何ですか？

関連情報: はじめに   良い材料を 早く創りたいので 、マテリアルズ・インフォマティクス（ MI）を活用して”いる”。このような声 はもう珍しくありません 。 MIブームに端を発して 、大学・企業 ともにMIの活用検討が 始められて 既にある程度の期間が 経過しました 。すでにMIに取り組んでいる 大学・企業 からはMI活用の成果 が多く報告されており 、MIは期待に応えられるということが共通理解になりつつあります。 MIとは？有効なものなのか？が大勢を占めていた数年前とは隔世の感があります。  本書を手に取っていただいた方は、 材料開発者として、 あるいは材料開発を支援する データ分析者として「MIを

回答: 

MI（Materials Informatics）は、**材料の特性や性能を理解し、設計・開発を効率化するための、材料科学とデータ分析の融合**です。

**具体的には、以下の要素が含まれます:**

* **材料のデータベース:**  膨大な材料データの蓄積と管理
* **データ分析:**  材料データから、材料の特性や性能を分析
* **AI・機械学習:**  材料の設計・開発を支援するAIや機械学習モデルの開発
* **シミュレーション:**  材料の性能を予測するためのシミュレーション

**MIのメリット:**

* **材料開発の効率化:**  従来の材料開発プロセスを大幅に短縮
* **新しい材料の発見:**  膨大なデータから、新しい材料の発見が可能
* **材料性能の向上:**  材料の性能をより正確に予測・設計
* **コスト削減:**  材料開発コストを削減


In [15]:
display(Markdown(ask_question(question)))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

質問: MIとは何ですか？

関連情報: はじめに   良い材料を 早く創りたいので 、マテリアルズ・インフォマティクス（ MI）を活用して”いる”。このような声 はもう珍しくありません 。 MIブームに端を発して 、大学・企業 ともにMIの活用検討が 始められて 既にある程度の期間が 経過しました 。すでにMIに取り組んでいる 大学・企業 からはMI活用の成果 が多く報告されており 、MIは期待に応えられるということが共通理解になりつつあります。 MIとは？有効なものなのか？が大勢を占めていた数年前とは隔世の感があります。  本書を手に取っていただいた方は、 材料開発者として、 あるいは材料開発を支援する データ分析者として「MIを

回答: 

MI（Materials Informatics）は、**材料の特性や性能を理解し、設計・開発を効率化するための、材料科学とデータ分析の融合**です。

**具体的には、以下の要素が含まれます:**

* **材料のデータベース:**  膨大な材料データの蓄積と管理
* **データ分析:**  材料データから、材料の特性や性能を分析
* **AI・機械学習:**  材料の設計・開発を支援するAIや機械学習モデルの開発
* **シミュレーション:**  材料の性能を予測するためのシミュレーション

**MIのメリット:**

* **材料開発の効率化:**  従来の材料開発プロセスを大幅に短縮
* **新しい材料の発見:**  膨大なデータから、新しい材料の発見が可能
* **材料性能の向上:**  材料の性能をより正確に予測・設計
* **コスト削減:**  材料開発コストを削減